# Deploy a training pipeline

## Before you start

Run the cell below to verify that all the required packages are installed.

In [ ]:
pip install -r "../requirements.txt"

### Connect to your workspace

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [1]:
from dotenv import load_dotenv
import os
load_dotenv() # take environment variables from .env.


subscription = os.environ["SUBSCRIPTION"]
resource_group = os.environ["RESOURCE_GROUP"]
ws_name = os.environ["WORKSPACE_NAME"]
compute_cluster = os.environ["COMPUTE_CLUSTER"]

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription,
    resource_group_name=resource_group,
    workspace_name=ws_name,
)

Create Data store and upload training data

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "../data"
dataset_name = "diabetes-data-train"

patient_dataset = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="Training data for diabetes prediction",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset)

In [3]:
patient_dataset = ml_client.data.get(
    name="diabetes-data-train", label="latest"
)

print(patient_dataset)

creation_context:
  created_at: '2024-12-10T13:58:08.309316+00:00'
  created_by: System Administrator
  created_by_type: User
  last_modified_at: '2024-12-10T13:58:08.336194+00:00'
description: Training data for diabetes prediction
id: /subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourceGroups/damen-pipeline-deployment-rg/providers/Microsoft.MachineLearningServices/workspaces/damen-pipeline-deployment-ws/data/diabetes-data-train/versions/9
name: diabetes-data-train
path: azureml://subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourcegroups/damen-pipeline-deployment-rg/workspaces/damen-pipeline-deployment-ws/datastores/workspaceblobstore/paths/LocalUpload/177b0cdcf5bcd30ff675ca95a93cbccd/data/
properties: {}
tags: {}
type: uri_folder
version: '9'



Load components

In [4]:
import os
from pathlib import Path

# Determine the project root relative to the current notebook's location
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # Adjust to go one level up to the project root

# Change the working directory to the project root
os.chdir(project_root)

from src.components import (
    remove_empty_rows, normalize_data, train_logistic_regression_classifier_model, train_decision_tree_classifier_model, compare_two_models)

In [ ]:
# ml_client.components.create_or_update(remove_empty_rows)
# ml_client.components.create_or_update(normalize_data)
# ml_client.components.create_or_update(train_logistic_regression_classifier_model)
# ml_client.components.create_or_update(train_decision_tree_classifier_model)
# ml_client.components.create_or_update(compare_two_models)


In [5]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification_py_func(pipeline_job_input):
    clean_data = remove_empty_rows(input_data=pipeline_job_input)
    normalized_data = normalize_data(input_data=clean_data.outputs.output_data)
    train_model_decision_tree = train_decision_tree_classifier_model(training_data=normalized_data.outputs.output_data)
    train_model_logistic_regression = train_logistic_regression_classifier_model(training_data=normalized_data.outputs.output_data)
    better_model = compare_two_models(model1=train_model_decision_tree.outputs.model_output_decision_tree, 
                                            model1_metrics=train_model_decision_tree.outputs.metrics_output,
                                            model2=train_model_logistic_regression.outputs.model_output_logistic_reg,
                                            model2_metrics=train_model_logistic_regression.outputs.metrics_output)                             
    
    return {
        "pipeline_job_transformed_data": normalized_data.outputs.output_data,
        "pipeline_job_trained_model_decision_tree": train_model_decision_tree.outputs.model_output_decision_tree,
        "pipeline_job_trained_model_logistic_regression": train_model_logistic_regression.outputs.model_output_logistic_reg,
        "pipeline_job_better_model": better_model.outputs.better_model,	
    }

pipeline_job = diabetes_classification_py_func(Input(type=AssetTypes.URI_FILE, path=patient_dataset.path))

print(pipeline_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


display_name: diabetes_classification_py_func
type: pipeline
inputs:
  pipeline_job_input:
    type: uri_file
    path: azureml://subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourcegroups/damen-pipeline-deployment-rg/workspaces/damen-pipeline-deployment-ws/datastores/workspaceblobstore/paths/LocalUpload/177b0cdcf5bcd30ff675ca95a93cbccd/data/
outputs:
  pipeline_job_transformed_data:
    type: uri_folder
  pipeline_job_trained_model_decision_tree:
    type: uri_folder
  pipeline_job_trained_model_logistic_regression:
    type: uri_folder
  pipeline_job_better_model:
    type: uri_folder
jobs:
  clean_data:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.pipeline_job_input}}
    component:
      name: remove_empty_rows
      display_name: Remove Empty Rows
      tags:
        codegenBy: mldesigner
      type: command
      inputs:
        input_data:
          type: uri_folder
      outputs:
        output_data:
          type: uri_folder
      comm

In [6]:
# set pipeline level compute
pipeline_job.settings.default_compute = compute_cluster
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes_training_py_func"
)
pipeline_job

Uploading src (0.02 MBs): 100%|##########| 17330/17330 [00:00<00:00, 84328.50it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
pipeline_diabetes_training_py_func,magenta_lemon_rwzxfzpwjs,pipeline,NotStarted,Link to Azure Machine Learning studio
